In [1]:
import pandas as pd 
import numpy as np
import csv
import os.path

In [18]:
#Importing the data from the csv file in the Data folder and turn them into pandas dataframe. Some of them needs to be converted to int datatype to combina all the datasets because some of the dataframe is type object when read'''
champions = pd.read_csv(r'./Data/champs.csv')
matches = pd.read_csv(r'./Data/matches.csv')
stats1 = pd.read_csv(r'./Data/stats1.csv')
stats12 = pd.read_csv(r'./Data/stats1-2.csv')
stats13 = pd.read_csv(r'./Data/stats1-3.csv')
stats2 = pd.read_csv(r'./Data/stats2.csv')
stats22 = pd.read_csv(r'./Data/stats2-2.csv')

#Appending stats1 and stats2 dataset
stats12 = stats12.append(stats13);
stats1 = stats1.append(stats12);
stats2 = stats2.append(stats22);
stats = stats1.append(stats2)

#Merge particpants with appropriate champion name. test
participants = pd.read_csv(r'./Data/participants.csv')

In [28]:
#Rename the column of the participants dataframe from id to play_id and championid to id for better clarification. inplace so no extra copy is made
participants.rename(columns ={'id': 'player_id','championid':'id'}, inplace = True)
#Merge the participants dataset with the champions dataset based on the championid. This is similar to left join in SQL 
champions['id'] = champions['id'].astype(float)
participants['id'] = participants['id'].astype(float)
participants = participants.merge(champions, how='left', on='id')

#Delete the column role because I think it's irrelevant for now 
#del participants['role']

#Rename the stats data set column from id to player_id and inplace 
stats.rename(columns={'id':'player_id'},inplace = True)
#Convert the player_id column to int because it is currently type Object and we cannot manipulate object type column and int type column too easily 
stats['player_id'] = stats['player_id'].astype(float)
#merge the participants dataset with stats dataset based on the player_id. This makes participants a massive dataset with lots of columns and champions that each player uses
participants = participants.merge(stats, how = 'left', on = 'player_id')

KeyError: 'id'

In [17]:
stats['firstblood'].describe()

count    1.834517e+06
mean     9.925392e-02
std      2.990027e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: firstblood, dtype: float64

In [ ]:
#This is me testing out the gorupby but this will group the champion based on their name and count how many win and lose they have. Win and lose are represented with 0 and 1 
champsFreq = participants.groupby(['name','win'])['win'].count()

#Turn it into a dataframe
champsFreq = pd.DataFrame(champsFreq)
#Add Suffix Count to the second column. When we groupby, it creates columns with same name and it's difficult to select columns so I add a letter to the column with the number of wins and lose for easy access to the dataframe
champsFreq = champsFreq.add_suffix('Count').reset_index()
#Converting the columns from object to int type
champsFreq['win'] = champsFreq['win'].astype(str).astype(int)

In [ ]:
#Separate the dataframe into two values, win and lose. Win is represented with 1 and lose is represented with 0
loseValues = champsFreq[champsFreq['win'] == 0]
winValues = champsFreq[champsFreq['win'] == 1] 

In [ ]:
print (stats1)

In [ ]:
stats['win'].describe()

In [ ]:
matches